In [1]:
import os
import numpy as np
import time
from PIL import Image
import utils

from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms
import torchvision.models as models

from efficientnet_pytorch import EfficientNet

from sklearn.metrics import f1_score, precision_recall_fscore_support

In [2]:
class customDataset(Dataset):
    def __init__(self, datatype, transform):
        ##############################################
        ### Initialize paths, transforms, and so on
        ##############################################
        self.transform = transform

        filename = open('./crawling/{}_list'.format(datatype), 'r')
        info = filename.readlines()
        self.images = [row.split(',')[0] for row in info]
        self.labels = [int(row.split(',')[1].strip()) for row in info]
        
        assert len(self.images) == len(self.labels), 'mismatched length!'
        print("image shape: {}, label shape: {}".format(len(self.images), len(self.labels)))
        
        
    def __getitem__(self, index):
        ##############################################
        # 1. Read from file (using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (torchvision.Transform).
        # 3. Return the data (e.g. image and label)
        ##############################################
        
        image = Image.open(self.images[index]).convert('RGB')
        label = self.labels[index]
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label
        
    def __len__(self):
        ##############################################
        ### Indicate the total size of the dataset
        ##############################################
        return len(self.images)

In [3]:
batch_size = 32
num_workers = 10
# min (100, 100), max (5792, 8688)
resize_size = (224, 224)
MODEL = 'efficientnet-b4'
# MODEL = 'CUSTOM'

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.5, contrast=0.5),
        # transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([
        transforms.Resize(resize_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]),
}

In [5]:
trainset = customDataset(datatype='train',
                         transform=data_transforms['train'])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

image shape: 57597, label shape: 57597


In [6]:
testset = customDataset(datatype='test',
                        transform=data_transforms['test'])
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

image shape: 0, label shape: 0


In [7]:
# for imgs, lbls in trainloader:
#     print('Size of image:', imgs.size())  
#     print('Type of image:', imgs.dtype)
#     print('Size of label:', lbls.size())  
#     print('Type of label:', lbls.dtype)
#     break

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
# PATH = './model/efficientnet-b4_20201012-132329_0.001_7_0.4532584269662922'
# net = utils.Net()
# net.load_state_dict(torch.load(PATH))
# net.to(device)

In [10]:
net = utils.Net()
net.to(device)

Loaded pretrained weights for efficientnet-b4


rnel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          28, 672, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding(
          672, 160, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (_bn2): BatchNorm2d(160, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_swish): MemoryEfficientSwish()
      )
      (17): MBConvBlock(
        (_expand_conv): Conv2dStaticSamePadding(
          160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (_bn0): BatchNorm2d(960, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_depthwise_conv): Conv2dStaticSamePadding(
          960, 960, kernel_size=(5, 5), stride=(1, 1), groups=960, bias=False
         

In [11]:
# freeze pretrained models
cnt = 0
print(len(list(net.modules())))
# for child in net.modules():
#     # cnt += 1
#     if cnt > 480:
#         print(child)
#         for param in child.parameters():
#             param.requires_grad = True
print(cnt)

489
0


In [12]:
# positive: 15433, negative: 41118
# positive: 17827, negative: 39770
class_weights = torch.FloatTensor([1, 2.5]).cuda()
# class_weights = torch.FloatTensor([1, 3]).cuda()
criterion = nn.CrossEntropyLoss(weight=class_weights)
epochs = 8
size = 46078 // batch_size
timestr = time.strftime("%Y%m%d-%H%M%S")

In [11]:
### First
LR = 1e-2
# optimizer = optim.Adam(net.parameters(), lr=1e-3)
optimizer = optim.SGD(net.parameters(), lr=LR)

for epoch in tqdm(range(epochs), desc='Epochs: '):  # loop over the dataset multiple times
    print("================== EPOCH INFO: {} ==================".format(epoch + 1))
    net.train()     # Train mode
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc='Progress:'), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        labels = labels.view(-1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        # print(data[0].shape, data[1].shape)
        # print(inputs.shape)
        # print(labels.shape, outputs.shape)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        interval = 500
        if i % interval == 0 and i != 0:    # print every interval
            _, y_pred_tag = torch.max(outputs, 1)
            correct_results_sum = (y_pred_tag == labels).sum().float()
            acc = correct_results_sum / labels.shape[0]

            y_pred_np = y_pred_tag.cpu().detach().numpy()
            y_true_np = labels.cpu().detach().numpy()
            print("==============================================")
            print("Batch acc = {} ({} / {})".format(acc, correct_results_sum, labels.shape[0]))
            print("Batch f1-score = {}".format(f1_score(y_true_np, y_pred_np, average='binary')))
            print(precision_recall_fscore_support(y_true_np, y_pred_np, average='binary'))

            print("Prediction distribution:")
            print(np.unique(y_pred_np, return_counts=True), np.unique(y_true_np, return_counts=True))
            # print(y_pred_np, y_true_np)

            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / interval))
            running_loss = 0.0

    # Eval mode
    net.eval()
    # Training score
    y_pred_train = np.array([])
    y_true_train = np.array([])
    with torch.no_grad():
        for data_test in tqdm(trainloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)

            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_train = np.hstack([y_pred_train, y_pred_tag.cpu().detach().numpy()])

            y_true_train = np.hstack([y_true_train, labels.cpu().detach().numpy()])

    precision_train, recall_train, f1_train, support_train = precision_recall_fscore_support(y_true_train, y_pred_train, average='binary')

    print()
    print("Training prediction:")
    print(np.unique(y_pred_train, return_counts=True), np.unique(y_true_train, return_counts=True))
    print(y_pred_train[0:10], y_true_train[0:10])
    print("F1 of model on train set: ", f1_score(y_true_train, y_pred_train, average='binary'))
    print("Precision: {}, Recall: {}, F1: {}".format(precision_train, recall_train, f1_train))

    # Testing score
    y_pred_test = np.array([])
    y_true_test = np.array([])
    with torch.no_grad():
        for data_test in tqdm(testloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)

            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_test = np.hstack([y_pred_test, y_pred_tag.cpu().detach().numpy()])
                
            y_true_test = np.hstack([y_true_test, labels.cpu().detach().numpy()])
    
    precision_test, recall_test, f1_test, support_test = precision_recall_fscore_support(y_true_test, y_pred_test, average='binary')

    print("Testing prediction:")
    print(np.unique(y_pred_test, return_counts=True), np.unique(y_true_test, return_counts=True))
    print(y_pred_test[0:10], y_true_test[0:10])
    print("F1 of my model on test set: ", f1_score(y_true_test, y_pred_test, average='binary'))
    print("Precision: {}, Recall: {}, F1: {}".format(precision_test, recall_test, f1_test))


    # record precision, recall, f1-score into file
    with open('record/{}_{}_train_loss'.format(MODEL, timestr), 'a') as f_train, open('record/{}_{}_test_loss'.format(MODEL, timestr), 'a') as f_test:
        f_train.write(str(precision_train) + ', ' + str(recall_train) + ', ' + str(f1_train) + '\n')
        f_test.write(str(precision_test) + ', ' + str(recall_test) + ', ' + str(f1_test) + '\n')

    # save model for every epoch
    torch.save(net.state_dict(), './model/{}_{}_{}_{}_{}_{}'.format(MODEL, timestr, LR, epoch, f1_train, f1_test))

================== EPOCH INFO: 1 ==================


KeyboardInterrupt: 

In [12]:
epochs = 2
# size = 46078 // batch_size
timestr = time.strftime("%Y%m%d-%H%M%S")

In [13]:
### Second try
LR = 1e-3
# optimizer = optim.Adam(net.parameters(), lr=1e-3)
optimizer = optim.SGD(net.parameters(), lr=LR)

for epoch in tqdm(range(epochs), desc='Epochs: '):  # loop over the dataset multiple times
    print("================== EPOCH INFO: {} ==================".format(epoch + 1))
    net.train()     # Train mode
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc='Progress:'), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        labels = labels.view(-1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        # print(data[0].shape, data[1].shape)
        # print(inputs.shape)
        # print(labels.shape, outputs.shape)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        interval = 500
        if i % interval == 0 and i != 0:    # print every interval
            _, y_pred_tag = torch.max(outputs, 1)
            correct_results_sum = (y_pred_tag == labels).sum().float()
            acc = correct_results_sum / labels.shape[0]

            y_pred_np = y_pred_tag.cpu().detach().numpy()
            y_true_np = labels.cpu().detach().numpy()
            # print("==============================================")
            # print("Batch acc = {} ({} / {})".format(acc, correct_results_sum, labels.shape[0]))
            # print("Batch f1-score = {}".format(f1_score(y_true_np, y_pred_np, average='binary')))
            # print(precision_recall_fscore_support(y_true_np, y_pred_np, average='binary'))

            # print("Prediction distribution:")
            # print(np.unique(y_pred_np, return_counts=True), np.unique(y_true_np, return_counts=True))
            # print(y_pred_np, y_true_np)

            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / interval))
            running_loss = 0.0

    # Eval mode
    net.eval()
    # Training score
    y_pred_train = np.array([])
    y_true_train = np.array([])
    with torch.no_grad():
        for data_test in tqdm(trainloader):
            images, labels = data_test[0].to(device), data_test[1].to(device)
            outputs = net(images)

            _, y_pred_tag = torch.max(outputs, 1)
            y_pred_train = np.hstack([y_pred_train, y_pred_tag.cpu().detach().numpy()])

            y_true_train = np.hstack([y_true_train, labels.cpu().detach().numpy()])

    precision_train, recall_train, f1_train, support_train = precision_recall_fscore_support(y_true_train, y_pred_train, average='binary')

    print()
    print("Training prediction:")
    print(np.unique(y_pred_train, return_counts=True), np.unique(y_true_train, return_counts=True))
    print(y_pred_train[0:10], y_true_train[0:10])
    print("F1 of model on train set: ", f1_score(y_true_train, y_pred_train, average='binary'))
    print("Precision: {}, Recall: {}, F1: {}".format(precision_train, recall_train, f1_train))

    # # Testing score
    # y_pred_test = np.array([])
    # y_true_test = np.array([])
    # with torch.no_grad():
    #     for data_test in tqdm(testloader):
    #         images, labels = data_test[0].to(device), data_test[1].to(device)
    #         outputs = net(images)

    #         _, y_pred_tag = torch.max(outputs, 1)
    #         y_pred_test = np.hstack([y_pred_test, y_pred_tag.cpu().detach().numpy()])
                
    #         y_true_test = np.hstack([y_true_test, labels.cpu().detach().numpy()])
    
    # precision_test, recall_test, f1_test, support_test = precision_recall_fscore_support(y_true_test, y_pred_test, average='binary')

    # print("Testing prediction:")
    # print(np.unique(y_pred_test, return_counts=True), np.unique(y_true_test, return_counts=True))
    # print(y_pred_test[0:10], y_true_test[0:10])
    # print("F1 of my model on test set: ", f1_score(y_true_test, y_pred_test, average='binary'))
    # print("Precision: {}, Recall: {}, F1: {}".format(precision_test, recall_test, f1_test))


    # record precision, recall, f1-score into file
    with open('record/{}_{}_train_loss'.format(MODEL, timestr), 'a') as f_train:
        f_train.write(str(precision_train) + ', ' + str(recall_train) + ', ' + str(f1_train) + '\n')

    # save model for every epoch
    torch.save(net.state_dict(), './model/{}_{}_{}_{}_{}'.format(MODEL, timestr, LR, epoch, f1_train))

================== EPOCH INFO: 1 ==================


[1,   501] loss: 0.719
[1,  1001] loss: 0.709
[1,  1501] loss: 0.705



Training prediction:
(array([0., 1.]), array([29037, 28560])) (array([0., 1.]), array([39770, 17827]))
[0. 0. 1. 1. 1. 1. 1. 1. 0. 1.] [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
F1 of model on train set:  0.3983012481945373
Precision: 0.3234593837535014, Recall: 0.5182027262018287, F1: 0.3983012481945373
================== EPOCH INFO: 2 ==================


[2,   501] loss: 0.702
[2,  1001] loss: 0.697
[2,  1501] loss: 0.698



Training prediction:
(array([0., 1.]), array([28529, 29068])) (array([0., 1.]), array([39770, 17827]))
[0. 1. 1. 0. 1. 1. 0. 0. 1. 0.] [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
F1 of model on train set:  0.4111312506663824
Precision: 0.33163616347873953, Recall: 0.5407527907107197, F1: 0.4111312506663824
================== EPOCH INFO: 3 ==================


...


Training prediction:
(array([0., 1.]), array([23173, 34424])) (array([0., 1.]), array([39770, 17827]))
[1. 0. 1. 1. 0. 1. 0. 1. 0. 0.] [0. 1. 1. 1. 0. 1. 0. 0. 0. 1.]
F1 of model on train set:  0.4419436948575147
Precision: 0.33540553102486637, Recall: 0.6476692657205363, F1: 0.4419436948575147
================== EPOCH INFO: 7 ==================


[7,   501] loss: 0.690
[7,  1001] loss: 0.692
[7,  1501] loss: 0.694



Training prediction:
(array([0., 1.]), array([21068, 36529])) (array([0., 1.]), array([39770, 17827]))
[0. 1. 1. 1. 1. 1. 1. 1. 0. 0.] [0. 0. 0. 1. 1. 0. 0. 1. 0. 0.]
F1 of model on train set:  0.4511001545367577
Precision: 0.33562375099236225, Recall: 0.6877208728333427, F1: 0.4511001545367577
================== EPOCH INFO: 8 ==================


[8,   501] loss: 0.692
[8,  1001] loss: 0.690
[8,  1501] loss: 0.690



Training prediction:
(array([0., 1.]), array([15047, 42550])) (array([0., 1.]), array([39770, 17827]))
[1. 1. 1. 1. 1. 0. 1. 0. 1. 1.] [1. 1. 0. 0. 0. 1. 0. 0. 0. 1.]
F1 of model on train set:  0.46570714013614456
Precision: 0.33041128084606347, Recall: 0.7886352162450216, F1: 0.46570714013614456


In [12]:
PATH = './model/efficientnet-b4_20201012-132329_0.001_7_0.4532584269662922'
new_net = utils.Net()
new_net.load_state_dict(torch.load(PATH))
new_net.to(device)
new_net.eval()

Loaded pretrained weights for efficientnet-b4


rnel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          28, 672, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding(
          672, 160, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (_bn2): BatchNorm2d(160, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_swish): MemoryEfficientSwish()
      )
      (17): MBConvBlock(
        (_expand_conv): Conv2dStaticSamePadding(
          160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False
          (static_padding): Identity()
        )
        (_bn0): BatchNorm2d(960, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_depthwise_conv): Conv2dStaticSamePadding(
          960, 960, kernel_size=(5, 5), stride=(1, 1), groups=960, bias=False
         

In [13]:
# Training score
y_pred_train = np.array([])
y_true_train = np.array([])
with torch.no_grad():
    for data_test in tqdm(trainloader):
        images, labels = data_test[0].to(device), data_test[1].to(device)
        outputs = new_net(images)

        _, y_pred_tag = torch.max(outputs, 1)
        y_pred_train = np.hstack([y_pred_train, y_pred_tag.cpu().detach().numpy()])

        y_true_train = np.hstack([y_true_train, labels.cpu().detach().numpy()])

precision_train, recall_train, f1_train, support_train = precision_recall_fscore_support(y_true_train, y_pred_train, average='binary')

print()
print("Training prediction:")
print(np.unique(y_pred_train, return_counts=True), np.unique(y_true_train, return_counts=True))
print(y_pred_train[0:10], y_true_train[0:10])
print("F1 of model on train set: ", f1_score(y_true_train, y_pred_train, average='binary'))
print("Precision: {}, Recall: {}, F1: {}".format(precision_train, recall_train, f1_train))


Training prediction:
(array([0., 1.]), array([22098, 35499])) (array([0., 1.]), array([39770, 17827]))
[1. 0. 0. 1. 1. 0. 0. 0. 1. 1.] [1. 0. 0. 1. 0. 0. 1. 0. 1. 1.]
F1 of model on train set:  0.45197464651389563
Precision: 0.33947435139018, Recall: 0.6759970830762326, F1: 0.45197464651389563


In [17]:
# Testing score
y_pred_test = np.array([])
y_true_test = np.array([])
with torch.no_grad():
    for data_test in tqdm(testloader):
        images, labels = data_test[0].to(device), data_test[1].to(device)
        outputs = new_net(images)

        _, y_pred_tag = torch.max(outputs, 1)
        y_pred_test = np.hstack([y_pred_test, y_pred_tag.cpu().detach().numpy()])
            
        y_true_test = np.hstack([y_true_test, labels.cpu().detach().numpy()])

precision_test, recall_test, f1_test, support_test = precision_recall_fscore_support(y_true_test, y_pred_test, average='binary')

print("Testing prediction:")
print(np.unique(y_pred_test, return_counts=True), np.unique(y_true_test, return_counts=True))
print(y_pred_test[0:10], y_true_test[0:10])
print("F1 of my model on test set: ", f1_score(y_true_test, y_pred_test, average='binary'))
print("Precision: {}, Recall: {}, F1: {}".format(precision_test, recall_test, f1_test))

Testing prediction:
(array([0., 1.]), array([3465, 8054])) (array([0., 1.]), array([7954, 3565]))
[1. 0. 1. 1. 1. 0. 1. 0. 1. 1.] [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
F1 of my model on test set:  0.44857560891643006
Precision: 0.3235659299726844, Recall: 0.7309957924263675, F1: 0.44857560891643006


In [2]:
new_net = utils.Net()

)
  (15): MBConvBlock(
    (_expand_conv): Conv2dStaticSamePadding(
      112, 672, kernel_size=(1, 1), stride=(1, 1), bias=False
      (static_padding): Identity()
    )
    (_bn0): BatchNorm2d(672, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_depthwise_conv): Conv2dStaticSamePadding(
      672, 672, kernel_size=(3, 3), stride=(1, 1), groups=672, bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn1): BatchNorm2d(672, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_se_reduce): Conv2dStaticSamePadding(
      672, 28, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_se_expand): Conv2dStaticSamePadding(
      28, 672, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_project_conv): Conv2dStaticSamePadding(
      672, 112, kernel_size=(1, 1), stride=(1, 1), bias=False
      (static_padding): Identity()
    )

In [3]:
new_net.layer_cnt

9